In [21]:
import os

#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

#graph deep learning
import torch
import torch_geometric
import networkx as nx

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch_geometric
%matplotlib inline

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# summarize
+ input_tensor: (time_num, nodes_dim * num_nodes)
+

# lstm train pipline

## import data
+ metric is input tensor shape: (time_num, nodes_dim * num_nodes)
+ every batch : `(t, b, n, c)`

In [22]:
from util.clean_data import get_and_normalize_data
import pickle

metric = get_and_normalize_data('DatasetUpdate/MMS (1).csv')
with open('DatasetUpdate/MMS_topology.pk', 'rb') as f:
    edge_index = pickle.load(f)

In [23]:
metric

pod    adservice-76469b5944-9tlwk                                      \
metric                CpuUsage(m) MemoryUsage(Mi) NetworkReceiveBytes   
0                       -0.157415        3.169402           -0.859508   
1                       -0.163589        3.169402           -0.860804   
2                       -0.128732        3.169402           -0.854620   
3                       -0.127304        3.169402           -0.860612   
4                       -0.124751        3.169402           -0.846356   
...                           ...             ...                 ...   
4365                     0.048746        2.971111           -0.653714   
4366                          NaN             NaN                 NaN   
4367                    -0.568951        2.972818           -0.733162   
4368                    -0.234948        2.971111           -0.723533   
4369                    -0.031994        2.971111           -0.717625   

pod                                                                           \
metric NetworkTransmitBytes PodLatency(s) PodSuccessRate(%) PodWorkload(Ops)   
0                 -0.539167     -0.265056         -0.322612        -1.293704   
1                 -0.554821     -0.265007         -0.840723        -1.259165   
2                 -0.558797     -0.265059         -0.281728        -1.278876   
3                 -0.559086     -0.264997         -0.935871        -1.278892   
4                 -0.546904     -0.265005         -0.863189        -1.263901   
...                     ...           ...               ...              ...   
4365              -0.447064     -0.264952          0.212791        -0.500626   
4366                    NaN           NaN               NaN              NaN   
4367              -0.507590     -0.264851         -0.352831        -0.849824   
4368              -0.499043     -0.264946         -0.371308        -0.756481   
4369              -0.485635     -0.265043          0.049564        -0.765983   

pod    adservice-76469b5944-h6cqg                                      ...  \
metric                CpuUsage(m) MemoryUsage(Mi) NetworkReceiveBytes  ...   
0                       -0.250438        3.018257           -0.862908  ...   
1                       -0.408388        3.018044           -0.862873  ...   
2                       -0.293421        3.018257           -0.864335  ...   
3                       -0.244242        3.018257           -0.862536  ...   
4                       -0.227894        3.018044           -0.858572  ...   
...                           ...             ...                 ...  ...   
4365                          NaN             NaN                 NaN  ...   
4366                     0.020721        3.200441           -0.655733  ...   
4367                    -0.367106        3.200654           -0.733405  ...   
4368                    -0.106638        3.200441           -0.712772  ...   
4369                    -0.373853        3.200441           -0.715206  ...   

pod    shippingservice-7499794f94-gmcld                                     \
metric                    PodLatency(s) PodSuccessRate(%) PodWorkload(Ops)   
0                             -0.252401          0.372415        -1.239457   
1                             -0.263458          0.372415        -1.254233   
2                             -0.231213          0.372415        -1.308469   
3                             -0.237058          0.372415        -1.343006   
4                             -0.259269          0.372415        -1.313256   
...                                 ...               ...              ...   
4365                                NaN               NaN              NaN   
4366                          -0.251305          0.372415        -0.594243   
4367                          -0.252714          0.372415        -0.829910   
4368                          -0.248383          0.372415        -0.855071   
4369                          -0.250209          0.372415 

## split train and test, then train

In [19]:
from sklearn.model_selection import train_test_split
from graph_lstm_vae_ad_ver6 import GraphLSTM_VAE_AD

train, test = train_test_split(metric, test_size=0.3, shuffle=False)

lstm = GraphLSTM_VAE_AD(variational=False)
lstm.fit(train, 50, edge_index)

  0%|          | 0/10 [00:00<?, ?it/s]


KeyboardInterrupt: 

## important process and data
In this section, I'll explain the architecture of this nn

### set sequence for train and test
+ train: 0.7 valid: 0.3
+ sequence shape: `(batch_size, sequence_length(known as timestep), nodes_num, performance_metrics_num)` in this table is (4341, 30, 50, 7)

in `graphlstm_vae_ad.py`

In [20]:
# sequence = lstm.get_time_sequence(metric.values, 50)
#
# from torch.utils.data import DataLoader, random_split
#
# trainset, validset = random_split(sequence, [0.7, 0.3])
# train_loader = DataLoader(dataset=trainset, batch_size=32, drop_last=True, shuffle=True, pin_memory=False)
# valid_loader = DataLoader(dataset=validset, batch_size=32, drop_last=True, shuffle=True, pin_memory=False)

### graphlstm_vae architecture
+ set encoder and decoder layer in `num_layers: tuple = (2, 2)`
+ every batch : `(t, b, n, c)`
+ set hidden dim  in `num_layers: int` but in graphlstm unit it transform to list `param = [param] * num_layers`

In [17]:
from graph_lstm_vae_ad_ver6 import GraphLSTM_VAE_AD

module = GraphLSTM_VAE_AD(num_layers=(3, 2), hidden_dim=6)
GraphLSTM_VAE = module.fit(metric, 50, edge_index)

  0%|          | 0/94 [00:01<?, ?it/s]


## encoder
compose of `self.num_layers[0]` conv nn
To use the encoder module, follow following shape
+  `ts_batch`: `(t, b, n, c)`

In [77]:
# encoder module

GraphLSTM(
  (cell_list): ModuleList(
    (0): GCNLSTMCell(
      (gconv): GCNConv(12, 20)
    )
    (1): GCNLSTMCell(
      (gconv): GCNConv(10, 20)
    )
  )
)

### GraphLSTM unit
+ h, c shape: (batch_size, num_nodes, num_hidden), (batch_size, num_nodes, num_hidden)

In [92]:
# call GraphLSTM unit forward will return h_next, c_next

### encoder output
call `_, enc_hidden0 = self.encoder(ts_batch.float(), edge_index)` get state

following code will generate encoder output:
+ layer_output_list shape: `(num_layers,batch_size, num_nodes, num_hidden)` h in this
+ layer_state_list shape: `(num_layers, [(batch_size, num_nodes, num_hidden), (batch_size, batch_size, num_nodes, num_hidden)])` (h, c) in this

In [ ]:
# for layer_idx in range(self.num_layers):
#
#     h, c = hidden_state[layer_idx]
#     output_inner = []
#     for t in range(seq_len):
#         h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[t],
#                                          edge_index=edge_index, cur_state=[h, c])
#         output_inner.append(h)
#
#     layer_output = torch.stack(output_inner, dim=0)
#     cur_layer_input = layer_output  # 4 dim
#
#     layer_output_list.append(layer_output)
#     last_state_list.append([h, c])
#
# if not self.return_all_layers:
#     layer_output_list = layer_output_list[-1:]
#     last_state_list = last_state_list[-1:]

## decoder without variational
+ first output shape: `(time_step, batch_size, num_nodes, nodes_dim)` in this table is (30, 32, 50, 7)

In [14]:
# get last output using nn.linear
# output[ts_batch.shape[0] - 1] = self.deal_list(self.hidden2output,
#                                                enc_hidden)

tensor([-0.8908,  0.5737, -1.1470,  0.0472,  0.6765,  0.3537, -0.1155],
       grad_fn=<SelectBackward0>)

### two way to train
1. using last output as input
2. using this time batch as input
+ `hidden_state` shape: `[[(b, n, h), (b, n, h)]] * num_layers` using encoder state as decoder hidden_state


In [ ]:
# for i in reversed(range(ts_batch.shape[0] - 1)):
#     if self.training and use_teacher_forcing:
#         _, dec_hidden = self.decoder(ts_batch[i + 1].unsqueeze(0).float(), edge_index, dec_hidden)
#     else:
#         _, dec_hidden = self.decoder(output[i + 1].unsqueeze(0), edge_index, dec_hidden)
#     output[i] = self.deal_list(self.hidden2output, dec_hidden[-1][
#         0])  # self.hidden2output(dec_hidden[-1][0])#self.deal_batch(self.hidden2output, dec_hidden[-1][0], edge_index)#self.hidden2output(dec_hidden[-1][0], edge_index)
